In [1]:
from abstractions.goap.spatial import GridMap, GameEntity, BlocksMovement, BlocksLight
from pydantic import ValidationError

# Create a grid map
grid_map = GridMap(width=10, height=10)

# Create some game entities
player = GameEntity(name="Player")
wall1 = GameEntity(name="Wall1", blocks_movement=BlocksMovement(value=True))
wall2 = GameEntity(name="Wall2", blocks_movement=BlocksMovement(value=True))
goal = GameEntity(name="Goal")

# Place entities on the grid map
grid_map.get_node((1, 1)).add_entity(player)
grid_map.get_node((3, 3)).add_entity(wall1)
grid_map.get_node((5, 5)).add_entity(wall2)
grid_map.get_node((8, 8)).add_entity(goal)

# Find the shortest path from the player's position to the goal using A*
start_node = grid_map.get_node(player.position.value)
goal_node = grid_map.get_node(goal.position.value)
path = grid_map.a_star(start_node, goal_node)

# Print the grid map with the path
print("Grid Map with A* Path:")
grid_map.print_grid(path)

# Find distances and paths from the player's position to all reachable nodes using Dijkstra
max_distance = 10
distances, paths = grid_map.dijkstra(start_node, max_distance)

# Print the distances and paths
print("\nDistances (Dijkstra):")
for node, distance in distances.items():
    print(f"Node at position {node}: Distance = {distance}")

print("\nPaths (Dijkstra):")
for node, path in paths.items():
    print(f"Node at position {node}: Path = {[n.position.value for n in path.nodes]}")

Grid Map with A* Path:
. . . . . . . . . . 
. S . . . . . . . . 
. . * . . . . . . . 
. . * # . . . . . . 
. . . * . . . . . . 
. . . . * # . . . . 
. . . . . * . . . . 
. . . . . . * . . . 
. . . . . . . * G . 
. . . . . . . . . . 

Distances (Dijkstra):
Node at position (1, 1): Distance = 0
Node at position (1, 2): Distance = 1
Node at position (1, 0): Distance = 1
Node at position (2, 1): Distance = 1
Node at position (0, 1): Distance = 1
Node at position (2, 2): Distance = 1
Node at position (0, 2): Distance = 1
Node at position (2, 0): Distance = 1
Node at position (0, 0): Distance = 1
Node at position (0, 3): Distance = 2
Node at position (1, 3): Distance = 2
Node at position (2, 3): Distance = 2
Node at position (3, 0): Distance = 2
Node at position (3, 1): Distance = 2
Node at position (3, 2): Distance = 2
Node at position (0, 4): Distance = 3
Node at position (1, 4): Distance = 3
Node at position (2, 4): Distance = 3
Node at position (3, 4): Distance = 3
Node at position (4, 0

In [2]:
from abstractions.goap.spatial import GridMap, GameEntity, BlocksMovement, BlocksLight
import random

def create_room(grid_map, top_left, width, height):
    for x in range(top_left[0], top_left[0] + width):
        for y in range(top_left[1], top_left[1] + height):
            grid_map.get_node((x, y)).reset()
            floor = GameEntity(name=f"Floor_{x}_{y}", blocks_movement=BlocksMovement(value=False), blocks_light=BlocksLight(value=False))
            grid_map.get_node((x, y)).add_entity(floor)

def create_h_corridor(grid_map, x1, x2, y):
    for x in range(min(x1, x2), max(x1, x2) + 1):
        grid_map.get_node((x, y)).reset()
        floor = GameEntity(name=f"Floor_{x}_{y}", blocks_movement=BlocksMovement(value=False), blocks_light=BlocksLight(value=False))
        grid_map.get_node((x, y)).add_entity(floor)

def create_v_corridor(grid_map, y1, y2, x):
    for y in range(min(y1, y2), max(y1, y2) + 1):
        grid_map.get_node((x, y)).reset()
        floor = GameEntity(name=f"Floor_{x}_{y}", blocks_movement=BlocksMovement(value=False), blocks_light=BlocksLight(value=False))
        grid_map.get_node((x, y)).add_entity(floor)

def generate_dungeon(grid_map, num_rooms, min_room_size, max_room_size):
    rooms = []
    for _ in range(num_rooms):
        width = random.randint(min_room_size, max_room_size)
        height = random.randint(min_room_size, max_room_size)
        x = random.randint(1, grid_map.width - width - 1)
        y = random.randint(1, grid_map.height - height - 1)
        create_room(grid_map, (x, y), width, height)
        rooms.append((x, y, width, height))
    for i in range(len(rooms) - 1):
        x1, y1, w1, h1 = rooms[i]
        x2, y2, w2, h2 = rooms[i + 1]
        if random.random() < 0.5:
            create_h_corridor(grid_map, x1 + w1, x2, y1 + h1 // 2)
            create_v_corridor(grid_map, y1 + h1 // 2, y2 + h2 // 2, x2)
        else:
            create_v_corridor(grid_map, y1 + h1 // 2, y2, x1 + w1 // 2)
            create_h_corridor(grid_map, x1 + w1 // 2, x2 + w2 // 2, y2)

# Create a larger grid map
grid_map = GridMap(width=50, height=50)

# Fill the grid map with walls
for x in range(grid_map.width):
    for y in range(grid_map.height):
        wall = GameEntity(name=f"Wall_{x}_{y}", blocks_movement=BlocksMovement(value=True), blocks_light=BlocksLight(value=True))
        grid_map.get_node((x, y)).add_entity(wall)

# Generate the dungeon
num_rooms = 10
min_room_size = 5
max_room_size = 10
generate_dungeon(grid_map, num_rooms, min_room_size, max_room_size)

# Find valid floor tiles for player and goal
floor_tiles = [(x, y) for x in range(grid_map.width) for y in range(grid_map.height) if any(isinstance(entity, GameEntity) and entity.name.startswith("Floor") for entity in grid_map.get_node((x, y)).entities)]
player_pos, goal_pos = random.sample(floor_tiles, 2)

# Place the player and goal entities
player = GameEntity(name="Player")
goal = GameEntity(name="Goal")
grid_map.get_node(player_pos).add_entity(player)
grid_map.get_node(goal_pos).add_entity(goal)

# Find the shortest path from the player's position to the goal using A*
start_node = grid_map.get_node(player.position.value)
goal_node = grid_map.get_node(goal.position.value)
path = grid_map.a_star(start_node, goal_node)
shadow = grid_map.get_shadow(start_node, max_radius=10)

# Print the grid map with the path
# grid_map.print_grid(path)
print("Grid Map with Shadow:")
grid_map.print_grid(shadow=shadow)

[(27, 13), (28, 12), (30, 9), (30, 18), (32, 12), (24, 17), (26, 14), (28, 14), (30, 11), (32, 14), (33, 13), (24, 10), (24, 19), (26, 16), (25, 18), (31, 15), (27, 8), (28, 7), (28, 16), (30, 13), (32, 7), (32, 16), (33, 15), (24, 12), (26, 9), (25, 11), (26, 18), (31, 8), (31, 17), (23, 13), (28, 9), (28, 18), (32, 9), (33, 8), (24, 14), (26, 11), (25, 13), (29, 13), (31, 10), (28, 11), (27, 15), (32, 11), (33, 10), (24, 16), (26, 13), (25, 15), (29, 15), (31, 12), (34, 11), (27, 17), (32, 13), (33, 12), (24, 9), (24, 18), (25, 17), (29, 8), (29, 17), (31, 14), (23, 10), (27, 10), (27, 19), (30, 15), (33, 14), (24, 11), (25, 10), (25, 19), (29, 10), (31, 7), (29, 19), (31, 16), (23, 12), (27, 12), (30, 8), (30, 17), (25, 12), (29, 12), (31, 9), (31, 18), (27, 14), (28, 13), (30, 10), (33, 9), (25, 14), (26, 15), (29, 14), (31, 11), (23, 16), (27, 16), (28, 15), (30, 12), (33, 11), (25, 16), (32, 15), (26, 17), (29, 16), (31, 13), (27, 9), (28, 8), (27, 18), (28, 17), (30, 14), (25, 9

In [3]:
print("Grid Map with A* Path:")
grid_map.print_grid(path=path)

Grid Map with A* Path:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # . . . . . . . . . . . . . . . . # # # # # # # # # # # # # # 
# # # # # # # # # # # # # # # # # # # # . # # # # # . . . . . . . . 

In [4]:
from abstractions.goap.spatial import GridMap, GameEntity, BlocksMovement, BlocksLight

# Create a grid map
grid_map = GridMap(width=20, height=20)

# Create walls for the room
for x in range(5, 15):
    for y in range(5, 15):
        if x == 5 or x == 14 or y == 5 or y == 14:
            wall = GameEntity(name=f"Wall_{x}_{y}", blocks_movement=BlocksMovement(value=True), blocks_light=BlocksLight(value=True))
            grid_map.get_node((x, y)).add_entity(wall)

# Create an opening in the room
opening_pos = (9, 5)
grid_map.get_node(opening_pos).entities.clear()

# Place the player entity inside the room
player_pos_inside = (9, 7)
player_inside = GameEntity(name="Player_Inside")
grid_map.get_node(player_pos_inside).add_entity(player_inside)

# Place the player entity at the boundary of the room
player_pos_boundary = (9, 5)
player_boundary = GameEntity(name="Player_Boundary")
grid_map.get_node(player_pos_boundary).add_entity(player_boundary)

# Place the player entity outside the room
player_pos_outside = (9, 3)
player_outside = GameEntity(name="Player_Outside")
grid_map.get_node(player_pos_outside).add_entity(player_outside)

# Test shadowcast inside the room
shadow_inside = grid_map.get_shadow(grid_map.get_node(player_pos_inside), max_radius=10)
print("Shadowcast inside the room:")
print(shadow_inside.nodes)
grid_map.print_grid(shadow=shadow_inside)

# Test shadowcast at the boundary of the room
shadow_boundary = grid_map.get_shadow(grid_map.get_node(player_pos_boundary), max_radius=10)
print("\nShadowcast at the boundary of the room:")
grid_map.print_grid(shadow=shadow_boundary)

# Test shadowcast outside the room
shadow_outside = grid_map.get_shadow(grid_map.get_node(player_pos_outside), max_radius=10)
print("\nShadowcast outside the room:")
grid_map.print_grid(shadow=shadow_outside)

# Test raycast inside the room
raycast_inside = grid_map.get_raycast(grid_map.get_node(player_pos_inside), grid_map.get_node((12, 12)))
print("\nRaycast inside the room:")
grid_map.print_grid(raycast=raycast_inside)

# Test raycast that should break (blocked by wall)
print("\nRaycast that should break (blocked by wall):")

try:
    raycast_break = grid_map.get_raycast(grid_map.get_node(player_pos_inside), grid_map.get_node((9, 19)))
    grid_map.print_grid(raycast=raycast_break)
except ValidationError as e:
    print(e)

[(8, 0), (5, 10), (8, 9), (10, 6), (9, 8), (11, 5), (11, 14), (13, 8), (6, 11), (7, 10), (8, 2), (9, 1), (8, 11), (9, 10), (11, 7), (13, 10), (6, 13), (7, 12), (14, 8), (9, 3), (9, 12), (11, 9), (13, 12), (6, 6), (7, 5), (7, 14), (12, 13), (14, 10), (5, 7), (9, 5), (9, 14), (13, 5), (13, 14), (7, 7), (12, 6), (14, 12), (5, 9), (9, 7), (10, 8), (13, 7), (7, 9), (12, 8), (9, 0), (5, 11), (9, 9), (10, 1), (8, 13), (10, 10), (13, 9), (7, 11), (12, 10), (14, 7), (9, 2), (5, 13), (8, 6), (10, 12), (13, 11), (11, 11), (6, 8), (12, 12), (14, 9), (5, 6), (8, 8), (10, 5), (10, 14), (11, 13), (6, 10), (12, 5), (12, 14), (14, 11), (8, 1), (8, 10), (10, 7), (11, 6), (6, 12), (12, 7), (10, 0), (8, 12), (10, 9), (9, 11), (11, 8), (6, 5), (6, 14), (7, 13), (12, 9), (14, 6), (8, 5), (10, 2), (9, 4), (8, 14), (10, 11), (9, 13), (11, 10), (13, 13), (6, 7), (7, 6), (12, 11), (5, 8), (8, 7), (9, 6), (10, 13), (11, 12), (13, 6), (6, 9), (7, 8)]
Shadowcast inside the room:
[Node(name='', id='3cc283de-c28d-41